In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import *
# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
import qiskit.ignis.mitigation.measurement as mc
from qiskit.quantum_info import state_fidelity


#Noise
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import QuantumError, ReadoutError
from qiskit.providers.aer.noise.errors import pauli_error, amplitude_damping_error
from qiskit.providers.aer.noise.errors import depolarizing_error, phase_damping_error
from qiskit.providers.aer.noise.errors import thermal_relaxation_error

from qiskit.ignis.characterization.coherence import T1Fitter, T2StarFitter, T2Fitter
from qiskit.ignis.characterization.coherence import t1_circuits, t2_circuits, t2star_circuits

In [3]:
# Create the expected density matrix
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.x(q2[0])
bell.ry(-np.pi/2,q2[1])
bell.cx(q2[1], q2[0])
print(bell)

job = qiskit.execute(bell, Aer.get_backend('statevector_simulator'))
psi_bell = job.result().get_statevector(bell)
print(psi_bell)

              ┌───┐     ┌───┐
q1_0: |0>─────┤ X ├─────┤ X ├
         ┌────┴───┴────┐└─┬─┘
q1_1: |0>┤ Ry(-1.5708) ├──■──
         └─────────────┘     
[ 0.        +0.j  0.70710678+0.j -0.70710678+0.j  0.        +0.j]


**Bit Flip:**

In [7]:
#Actual Circuit
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.x(q2[0])
bell.ry(-np.pi/2,q2[1])
bell.cx(q2[1], q2[0])

# Generate the state tomography circuits.
qst_bell = state_tomography_circuits(bell, [q2[0],q2[1]])
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_bell = StateTomographyFitter(job.result(), qst_bell)

# Perform the tomography fit
# which outputs a density matrix
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)

In [10]:
p_meas = 0.1
p_gate1 = 0.1

# QuantumError objects
##error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
error_gate2 = error_gate1.tensor(error_gate1)

# Add errors to noise model
noise_bit_flip = NoiseModel()
##noise_bit_flip.add_all_qubit_quantum_error(error_reset, "reset")
noise_bit_flip.add_all_qubit_quantum_error(error_meas, "measure")
noise_bit_flip.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
noise_bit_flip.add_all_qubit_quantum_error(error_gate2, ["cx"])

meas_calibs, state_labels = mc.complete_meas_cal(qubit_list=[0,1])

backend = Aer.get_backend('qasm_simulator')
job_cal = qiskit.execute(meas_calibs, backend=backend, shots=15000, noise_model=noise_bit_flip)
job_tomo = qiskit.execute(qst_bell, backend=backend, shots=15000, noise_model=noise_bit_flip)

meas_fitter = mc.CompleteMeasFitter(job_cal.result(),state_labels)

tomo_bell = StateTomographyFitter(job_tomo.result(), qst_bell)

#no correction
rho_bell1 = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell1)
print('Fit Fidelity (no correction) =', F_bell)
print(rho_bell1)


Fit Fidelity (no correction) = 0.6136155314392271
[[ 0.14798657+0.j          0.00264287-0.00092474j  0.00082503+0.0022418j
  -0.05543277-0.00427123j]
 [ 0.00264287+0.00092474j  0.35581423+0.j         -0.26138121-0.00400399j
  -0.00058057+0.00125377j]
 [ 0.00082503-0.0022418j  -0.26138121+0.00400399j  0.34865442+0.j
  -0.00342533-0.00311451j]
 [-0.05543277+0.00427123j -0.00058057-0.00125377j -0.00342533+0.00311451j
   0.14754479+0.j        ]]


**Phase Flip:**

In [18]:
#Actual Circuit
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.x(q2[0])
bell.ry(-np.pi/2,q2[1])
bell.cx(q2[1], q2[0])
#bell.ry(-np.pi/2,q2[1])
#bell.ry(-np.pi/2,q2[0])

# Generate the state tomography circuits.
qst_bell = state_tomography_circuits(bell, [q2[0],q2[1]])
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_bell = StateTomographyFitter(job.result(), qst_bell)

# Perform the tomography fit
# which outputs a density matrix
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)

In [19]:
p_meas = 0.1
p_gate1 = 0.1

# QuantumError objects
##error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
error_meas = pauli_error([('Z',p_meas), ('I', 1 - p_meas)])
error_gate1 = pauli_error([('Z',p_gate1), ('I', 1 - p_gate1)])
error_gate2 = error_gate1.tensor(error_gate1)

# Add errors to noise model
noise_phase_flip = NoiseModel()
##noise_bit_flip.add_all_qubit_quantum_error(error_reset, "reset")
noise_phase_flip.add_all_qubit_quantum_error(error_meas, "measure")
noise_phase_flip.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
noise_phase_flip.add_all_qubit_quantum_error(error_gate2, ["cx"])

meas_calibs, state_labels = mc.complete_meas_cal(qubit_list=[0,1])

backend = Aer.get_backend('qasm_simulator')
job_cal = qiskit.execute(meas_calibs, backend=backend, shots=15000, noise_model=noise_phase_flip)
job_tomo = qiskit.execute(qst_bell, backend=backend, shots=15000, noise_model=noise_phase_flip)

meas_fitter = mc.CompleteMeasFitter(job_cal.result(),state_labels)

tomo_bell = StateTomographyFitter(job_tomo.result(), qst_bell)

#no correction
rho_bell1 = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell1)
print('Fit Fidelity (no correction) =', F_bell)
print(rho_bell1)


Fit Fidelity (no correction) = 0.7551108526360641
[[ 8.51212493e-04+0.j         -2.79746633e-03+0.00121475j
   7.32668691e-04+0.00186112j -2.08997091e-04+0.00080622j]
 [-2.79746633e-03-0.00121475j  5.00120198e-01+0.j
  -2.55982498e-01+0.00278092j -3.59325560e-03-0.00140815j]
 [ 7.32668691e-04-0.00186112j -2.55982498e-01-0.00278092j
   4.98136511e-01+0.j          6.76080871e-03+0.00098111j]
 [-2.08997091e-04-0.00080622j -3.59325560e-03+0.00140815j
   6.76080871e-03-0.00098111j  8.92077809e-04+0.j        ]]


**Depolarization:**

In [12]:
#Actual Circuit
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.x(q2[0])
bell.ry(-np.pi/2,q2[1])
bell.cx(q2[1], q2[0])

# Generate the state tomography circuits.
qst_bell = state_tomography_circuits(bell, [q2[0],q2[1]])
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_bell = StateTomographyFitter(job.result(), qst_bell)

# Perform the tomography fit
# which outputs a density matrix
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)

In [13]:
noise_model = NoiseModel()

error = depolarizing_error(0.1, 1)
error2 = error.tensor(error)
noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(error2, ["cx"])
    
#generate the calibration circuits
meas_calibs, state_labels = mc.complete_meas_cal(qubit_list=[0,1])

backend = Aer.get_backend('qasm_simulator')
job_cal = qiskit.execute(meas_calibs, backend=backend, shots=15000, noise_model=noise_model)
job_tomo = qiskit.execute(qst_bell, backend=backend, shots=15000, noise_model=noise_model)

meas_fitter = mc.CompleteMeasFitter(job_cal.result(),state_labels)

tomo_bell = StateTomographyFitter(job_tomo.result(), qst_bell)

#no correction
rho_bell1 = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell1)
print('Fit Fidelity (no correction) =', F_bell)
print(rho_bell1)

Fit Fidelity (no correction) = 0.8180798369511377
[[ 0.04601994+0.j         -0.00098213-0.00063986j -0.00332285-0.00380295j
  -0.00174105+0.00569184j]
 [-0.00098213+0.00063986j  0.45394486+0.j         -0.3631825 -0.00063966j
   0.00460348-0.00091826j]
 [-0.00332285+0.00380295j -0.3631825 +0.00063966j  0.4558498 +0.j
  -0.00189922-0.00010738j]
 [-0.00174105-0.00569184j  0.00460348+0.00091826j -0.00189922+0.00010738j
   0.0441854 +0.j        ]]


**Phase Damping:**

In [20]:
#Actual Circuit
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.x(q2[0])
bell.ry(-np.pi/2,q2[1])
bell.cx(q2[1], q2[0])

# Generate the state tomography circuits.
qst_bell = state_tomography_circuits(bell, [q2[0],q2[1]])
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_bell = StateTomographyFitter(job.result(), qst_bell)

# Perform the tomography fit
# which outputs a density matrix
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)

In [21]:
noise_model = NoiseModel()

error = phase_damping_error(0.1, 1)
error2 = error.tensor(error)
noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(error2, ["cx"])
    
#generate the calibration circuits
meas_calibs, state_labels = mc.complete_meas_cal(qubit_list=[0,1])

backend = Aer.get_backend('qasm_simulator')
job_cal = qiskit.execute(meas_calibs, backend=backend, shots=15000, noise_model=noise_model)
job_tomo = qiskit.execute(qst_bell, backend=backend, shots=15000, noise_model=noise_model)

meas_fitter = mc.CompleteMeasFitter(job_cal.result(),state_labels)

tomo_bell = StateTomographyFitter(job_tomo.result(), qst_bell)

#no correction
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity (no correction) =', F_bell)
print(rho_bell)

Fit Fidelity (no correction) = 0.9228707235340919
[[ 8.18365448e-04+0.j         -8.37148347e-04+0.00032572j
  -2.39954029e-03-0.00311354j  3.05217930e-04+0.00051583j]
 [-8.37148347e-04-0.00032572j  5.00712818e-01+0.j
  -4.23671910e-01+0.00067098j  7.21743692e-04+0.00039469j]
 [-2.39954029e-03+0.00311354j -4.23671910e-01-0.00067098j
   4.97684809e-01+0.j          2.15605945e-03+0.00147846j]
 [ 3.05217930e-04-0.00051583j  7.21743692e-04-0.00039469j
   2.15605945e-03-0.00147846j  7.84008193e-04+0.j        ]]


**Amplitude Damping:**

In [22]:
#Actual Circuit
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.x(q2[0])
bell.ry(-np.pi/2,q2[1])
bell.cx(q2[1], q2[0])

# Generate the state tomography circuits.
qst_bell = state_tomography_circuits(bell, [q2[0],q2[1]])
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_bell = StateTomographyFitter(job.result(), qst_bell)

# Perform the tomography fit
# which outputs a density matrix
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)

In [23]:
noise_model = NoiseModel()

error = amplitude_damping_error(0.1, 0, True)
error2 = error.tensor(error)
noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(error2, ["cx"])
    
#generate the calibration circuits
meas_calibs, state_labels = mc.complete_meas_cal(qubit_list=[0,1])

backend = Aer.get_backend('qasm_simulator')
job_cal = qiskit.execute(meas_calibs, backend=backend, shots=15000, noise_model=noise_model)
job_tomo = qiskit.execute(qst_bell, backend=backend, shots=15000, noise_model=noise_model)

meas_fitter = mc.CompleteMeasFitter(job_cal.result(),state_labels)

tomo_bell = StateTomographyFitter(job_tomo.result(), qst_bell)

#no correction
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity (no correction) =', F_bell)
print(rho_bell)


Fit Fidelity (no correction) = 0.8764231778333289
[[ 9.91726908e-02+0.00000000e+00j -3.33443861e-03+3.17957807e-03j
   3.70137348e-05+5.13825444e-05j  2.12698460e-04+3.00937656e-03j]
 [-3.33443861e-03-3.17957807e-03j  4.94596059e-01+0.00000000e+00j
  -4.26273874e-01+2.56098899e-03j -4.20785050e-04-2.38455039e-03j]
 [ 3.70137348e-05-5.13825444e-05j -4.26273874e-01-2.56098899e-03j
   4.05702550e-01+0.00000000e+00j  3.33042011e-03-8.31645648e-04j]
 [ 2.12698460e-04-3.00937656e-03j -4.20785050e-04+2.38455039e-03j
   3.33042011e-03+8.31645648e-04j  5.28700729e-04+0.00000000e+00j]]


**Thermal Relaxation:**

In [25]:
#Actual Circuit
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.x(q2[0])
bell.ry(-np.pi/2,q2[1])
bell.cx(q2[1], q2[0])

# Generate the state tomography circuits.
qst_bell = state_tomography_circuits(bell, [q2[0],q2[1]])
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_bell = StateTomographyFitter(job.result(), qst_bell)

# Perform the tomography fit
# which outputs a density matrix
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)

In [26]:
# T1 and T2 values for qubits 0-3
T1s = np.random.normal(50e3, 10e3, 4) # Sampled from normal distribution mean 50 microsec
T2s = np.random.normal(70e3, 10e3, 4)  # Sampled from normal distribution mean 50 microsec

# Truncate random T2s <= T1s
T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(4)])

# Instruction times (in nanoseconds)
time_u1 = 0   # virtual gate
time_u2 = 50  # (single X90 pulse)
time_u3 = 100 # (two X90 pulses)
time_cx = 300
time_reset = 1000  # 1 microsecond
time_measure = 1000 # 1 microsecond

# QuantumError objects
errors_reset = [thermal_relaxation_error(t1, t2, time_reset)
                for t1, t2 in zip(T1s, T2s)]
errors_measure = [thermal_relaxation_error(t1, t2, time_measure)
                  for t1, t2 in zip(T1s, T2s)]
errors_u1  = [thermal_relaxation_error(t1, t2, time_u1)
              for t1, t2 in zip(T1s, T2s)]
errors_u2  = [thermal_relaxation_error(t1, t2, time_u2)
              for t1, t2 in zip(T1s, T2s)]
errors_u3  = [thermal_relaxation_error(t1, t2, time_u3)
              for t1, t2 in zip(T1s, T2s)]
errors_cx = [[thermal_relaxation_error(t1a, t2a, time_cx).expand(
             thermal_relaxation_error(t1b, t2b, time_cx))
              for t1a, t2a in zip(T1s, T2s)]
               for t1b, t2b in zip(T1s, T2s)]

# Add errors to noise model
noise_thermal = NoiseModel()
for j in range(4):
    noise_thermal.add_quantum_error(errors_reset[j], "reset", [j])
    noise_thermal.add_quantum_error(errors_measure[j], "measure", [j])
    noise_thermal.add_quantum_error(errors_u1[j], "u1", [j])
    noise_thermal.add_quantum_error(errors_u2[j], "u2", [j])
    noise_thermal.add_quantum_error(errors_u3[j], "u3", [j])
    for k in range(4):
        noise_thermal.add_quantum_error(errors_cx[j][k], "cx", [j, k])

#generate the calibration circuits
meas_calibs, state_labels = mc.complete_meas_cal(qubit_list=[0,1])

backend = Aer.get_backend('qasm_simulator')
job_cal = qiskit.execute(meas_calibs, backend=backend, shots=15000, noise_model=noise_model)
job_tomo = qiskit.execute(qst_bell, backend=backend, shots=15000, noise_model=noise_model)

meas_fitter = mc.CompleteMeasFitter(job_cal.result(),state_labels)

tomo_bell = StateTomographyFitter(job_tomo.result(), qst_bell)

#no correction
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity (no correction) =', F_bell)
print(rho_bell)


Fit Fidelity (no correction) = 0.8777335180304411
[[ 9.87709363e-02+0.j          1.00102830e-03+0.0017422j
  -1.43075238e-03-0.00044185j  1.27080066e-03-0.00305389j]
 [ 1.00102830e-03-0.0017422j   4.97769847e-01+0.j
  -4.27212580e-01-0.00259983j -8.22112351e-04+0.00043435j]
 [-1.43075238e-03+0.00044185j -4.27212580e-01+0.00259983j
   4.03272028e-01+0.j          6.87070353e-04+0.00128036j]
 [ 1.27080066e-03+0.00305389j -8.22112351e-04-0.00043435j
   6.87070353e-04-0.00128036j  1.87188587e-04+0.j        ]]
